<a href="https://colab.research.google.com/github/alfa7g7/Analisis-Cuantitativo-1er-semestre/blob/main/Sesion1/1-spacy-basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Básico con Spacy

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-nlp/blob/main/Sesion1/1-spacy-basics.ipynb)

## Referencias
* [NLP - Natural Language Processing With Python](https://www.udemy.com/course/nlp-natural-language-processing-with-python)
* [Natural Language Processing in Action](https://www.manning.com/books/natural-language-processing-in-action)

Este notebook contiene ejemplos básico de uso de la librería Spacy para procesamiento de lenguaje natural con técnicas clásicas. Esta herramienta nos servirá para familiarizarnos con los métodos clásicos.

## Preparación del entorno
Asumiendo que la librería ya se encuentra instalada, dependiendo de la tarea, necesitamos descargar un corpus, por ejemplo en el idioma ingles sería:

In [6]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

In [2]:
!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/requirements.txt && pip install -r requirements.txt

--2026-02-07 14:39:09--  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/requirements.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Ohtar10/icesi-nlp/refs/heads/main/requirements.txt [following]
--2026-02-07 14:39:09--  https://raw.githubusercontent.com/Ohtar10/icesi-nlp/refs/heads/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 349 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     349  --.-KB/s    in 0s      

2026-02-07 14:39:09 (9.43 MB/s) - ‘requirements.txt’ saved [349/349]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 116.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


El cual debemos luego importar:

In [1]:
import spacy

# load the simplified version of the english core language
nlp = spacy.load('en_core_web_sm')

## Creando un documento simple
Este documento será automáticamente interpretado con spacy para el lenguaje seleccionado.

In [2]:
doc = nlp(u'Tesla is looking at buying U.S. startup for $6 million')

Desde aquí, podemos observar los diferentes elementos del documento.

In [3]:
col1 = "Token"
col2 = "POS" # Part of Speech
col3 = "S-dep" # Syntactic dependency

print(f"{col1:{20}}{col2:{20}}{col3:{20}}")
for token in doc:
    print(f"{token.text:{20}}{token.pos_:{20}}{token.dep_}")

Token               POS                 S-dep               
Tesla               PROPN               nsubj
is                  AUX                 aux
looking             VERB                ROOT
at                  ADP                 prep
buying              VERB                pcomp
U.S.                PROPN               dobj
startup             VERB                advcl
for                 ADP                 prep
$                   SYM                 quantmod
6                   NUM                 compound
million             NUM                 pobj


Hemos impreso los tokens (palabras en este caso), la parte del contexto que representan (POS) y la dependencia semantica que dicho token tiene.

En NLP clásico hay una taxonomía especializada para cada elemento del lenguaje. Cada elemento fue producto de estudios diversos y variados con el fin de ofrecer un modelado sistemático del lenguaje. Expertos en lenguaje estuvieron involucrados en la creación de esta taxonomía.

Ahora, librerías como Spacy facilitan el procesamiento de esta taxonomía.

## Un pipeline simple de Spacy

El núcleo de Spacy es el pipeline que no es más que el procesamiento/transformación que toma el texto original y se lo somete a diversos procesos de NLP

In [4]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x787e403a7650>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x787e0c1d64b0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x787e0c253a70>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x787e002edd90>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x787e00102910>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x787e0c253a00>)]

Como podemos observar aquí, la instanciación por defecto es un pipeline compuesto por diferentes componentes que deberían ser familiares para nosotros:

* Token 2 Vec: Convertir los tokens en vectores.
* Lemmatizer: Extracción de componentes raíz de las palabras
* NER: Named entity recognition para identificar los sujetos de los documentos.

Un documento es iterable y los items pueden ser accedidos por índice.

In [5]:
n = 0
print(f"The {n}th token in the document is: {doc[n]}")

The 0th token in the document is: Tesla


## Exploremos diferentes elementos transformados

In [6]:
from spacy.tokens.doc import Doc
import pandas as pd

def get_doc_elements(doc: Doc):
    elements = ["text", "lemma", "pos", "tag", "shape", "alpha", "stop"]
    rows = [ [token.text, token.lemma_, token.pos_, token.tag_, token.shape_, token.is_alpha, token.is_stop]
            for token in  doc]
    return pd.DataFrame(rows, columns=elements)

In [7]:
doc_elements = get_doc_elements(doc)
doc_elements

,text,lemma,pos,tag,shape,alpha,stop
0,Tesla,Tesla,PROPN,NNP,Xxxxx,True,False
1,is,be,AUX,VBZ,xx,True,True
2,looking,look,VERB,VBG,xxxx,True,False
3,at,at,ADP,IN,xx,True,True
4,buying,buy,VERB,VBG,xxxx,True,False
5,U.S.,U.S.,PROPN,NNP,X.X.,False,False
6,startup,startup,VERB,VBD,xxxx,True,False
7,for,for,ADP,IN,xxx,True,True
8,$,$,SYM,$,$,False,False
9,6,6,NUM,CD,d,False,False


Done:

|Tag|Descrición|doc2[0].tag|
|:------|:------:|:------|
|`.text`|The original word text<!-- .element: style="text-align:left;" -->|`Tesla`|
|`.lemma_`|The base form of the word|`tesla`|
|`.pos_`|The simple part-of-speech tag|`PROPN`/`proper noun`|
|`.tag_`|The detailed part-of-speech tag|`NNP`/`noun, proper singular`|
|`.shape_`|The word shape – capitalization, punctuation, digits|`Xxxxx`|
|`.is_alpha`|Is the token an alpha character?|`True`|
|`.is_stop`|Is the token part of a stop list, i.e. the most common words of the language?|`False`|

## Objetos Span
Un span puede interpretarse como una porción de un documento, es decir, puede empezar desde alún índice hasta otro. Esto facilita el procesamiento por pedazos (chunks) en lugar el documento completo.

In [8]:
# Definition of NLP according to Wikipedia
doc = nlp(u"Natural language processing (NLP) is a subfield of computer science, \
information engineering, and artificial intelligence concerned with the \
interactions between computers and human (natural) languages, in particular \
how to program computers to process and analyze large amounts of natural language data.\
Challenges in natural language processing frequently involve speech recognition, natural \
language understanding, and natural language generation.")

quote = doc[10:30]
quote

computer science, information engineering, and artificial intelligence concerned with the interactions between computers and human (natural)

Observemos aquí que el slice es por los tokens y no por los caracteres individuales. Esto es muy útil ya que podemos estar seguros de no interrumpir abruptamente los tokens.

## Trabajando con oraciones
Podemos iterar sobre oraciones en los documentos, es decir, frases separadas por el punto "."

In [9]:
doc = nlp("This is the first sentence. This is the second sentence. And this is the last sentence.")
for sent in doc.sents:
    print(sent)

This is the first sentence.
This is the second sentence.
And this is the last sentence.


**Nota:** Cada punto es considerado un token, etnonces en el segundo "This" en el anterior documento está en el índice `6`, no en el `5`.

In [10]:
print(f"Token 5: {doc[5]}")
print(f"Token 6: {doc[6]}")
print(f"Is token 6 a sentence start? {doc[6].is_sent_start}")

Token 5: .
Token 6: This
Is token 6 a sentence start? True
